In [ ]:
# Librerias utiles
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import plotly
import seaborn as sns 
import plotly.express as px
import missingno as msno
import plotly.io as pio
import plotly.graph_objects as go
from mapsmx import MapsMX
import geopandas
import pyproj
import json
sns.set_theme()

In [ ]:
state = MapsMX().get_geo('state')
muns = MapsMX().get_geo('municipality')
yuc = muns[muns['cve_ent']=='31']

In [ ]:
feminicidios = pd.read_csv('../BD-sucias/feminicidios-corregida.csv', low_memory=False)
feminicidios.fecha_recepcion = pd.to_datetime(feminicidios.fecha_recepcion, format='%d/%m/%Y', errors='ignore')
feminicidios['año_recepcion'] = feminicidios['fecha_recepcion'].dt.year
nas = feminicidios[(feminicidios.TipoRelacion.isna()==True)|(feminicidios.TipoRelacion=='Seleccione')].index
feminicidios.loc[nas,'TipoRelacion'] = 'Desconocido'
com = feminicidios[(feminicidios.TipoRelacion=='En la comunidad')].index
feminicidios.loc[com,'TipoRelacion'] = 'Comunidad'
#euv_feminicidios = feminicidios.fk_euv.unique()

In [ ]:
servicios = pd.read_csv('../BD-sucias/servicios_18_OCTUBRE_2021.csv', low_memory=False)

In [ ]:
victimas = pd.read_csv('reportes/reporte_semujeres_18_OCTUBRE_2021.csv',low_memory=False, dtype={'pk_perfil_agresor': 'object','num_hijos':'int32'}, parse_dates=['fecha_recepcion','fecha_hechos'])

In [ ]:
subtipo = pd.read_csv('../BD-sucias/subtipo_18_OCTUBRE_2021.csv', low_memory=False)
subtipo.fecha_recepcion = pd.to_datetime(subtipo.fecha_recepcion, format='%d/%m/%Y', errors='ignore')
subtipo.fecha_hechos = pd.to_datetime(subtipo.fecha_hechos, format='%d/%m/%Y', errors='ignore')
subtipo = subtipo.dropna()
selecciones = subtipo[subtipo.SubtipoOrd=='Seleccione'].index
subtipo.loc[selecciones, 'SubtipoOrd'] = 'No especificado'

## Casos de violencia hacia las mujeres por tipo de violencia (Estatal/Municipal)

In [ ]:
victimas = pd.concat([victimas, victimas.tipos.str.get_dummies(sep=',')], axis=1)

In [ ]:
victimas['Feminicida']=0
fems = victimas[victimas['descripcion_otro_tipos']=='FEMINICIDA'].index
victimas.loc[fems,'Feminicida'] = 1
victimas.loc[fems,'Otro'] = 0

In [ ]:
victimas.info()

In [ ]:
tipos = victimas.iloc[:,[i for i in range(42,49)]]
tipos = pd.DataFrame(tipos.sum())
tipos = tipos.sort_values(0,ascending=False)
#new_row = pd.Series(data={0:sum(victimas.descripcion_otro_tipos=='FEMINICIDA')}, name='Feminicida')
#append row to the dataframe
#tipos = tipos.append(new_row)
#tipos.loc['Otro',0] = tipos.loc['Otro',0] - tipos.loc['Feminicida',0]
tipos = tipos.sort_values(0, ascending=False)
y = tipos[0].values
y_total = len(victimas)
fig = px.bar(x=tipos.index,
            y=tipos[0],
            text= np.round(y/y_total*100,2),
            labels = {'x': 'Tipo de violencia', "y":'Número de casos', 'text':'Porcentaje'},
            color=px.colors.qualitative.Prism[:7],
            color_discrete_map="identity",
            title='Gráfica 1. Casos de violencia hacia las mujeres por tipo de violencia'
            )
fig.update_xaxes(type='category')
fig.update_layout( xaxis_title=None,  height=500)
fig.update_traces(texttemplate='%{text} %')
#fig.update_xaxes(visible=False, showticklabels=False)
pio.write_html(fig, file='../docs/figs/indicador_municipal_barras_violencias.html.html',include_plotlyjs='cdn', full_html=False)
#print(fig.data[0].hovertemplate)
fig.update_traces(hovertemplate='Tipo de violencia=%{x}<br>Número de casos=%{y}<br>Porcentaje=%{text} %<extra></extra>')
fig.show()

In [ ]:
victimas_2020 = victimas[victimas.año_recepcion==2020]

In [ ]:
victimas_2019 = victimas[victimas.año_recepcion==2019]

In [ ]:
# censo poblacion y vivienda 2020 inegi
inegi_2020 = pd.read_csv('censo_yuc.csv')
pob_fem = inegi_2020[inegi_2020.NOM_LOC=='Total del Municipio'][['MUN','NOM_MUN','POBFEM' ]].reset_index(drop=True)

In [ ]:
feminicidios = feminicidios.merge(victimas[['fk_euv','Edad Agresor']], left_on='fk_euv', right_on='fk_euv', how='left')
feminicidios = feminicidios.drop_duplicates(keep='last')
feminicidios = feminicidios.reset_index(drop=True)

In [ ]:
feminicidios_sent = feminicidios[feminicidios.Sentencia.str.contains("TENTATIVA")==False]
tentativas_sent = feminicidios[feminicidios.Sentencia.str.contains("TENTATIVA")==True]

In [ ]:
fem_mun = feminicidios_sent.municipioV.value_counts()
fem_mun = yuc[['geometry_mun','nom_mun']].merge(fem_mun, right_on=fem_mun.index, left_on='nom_mun', how='left')
fem_mun.fillna(0,inplace=True)
fem_mun.municipioV = fem_mun.municipioV.astype(int)

fem_mun_tent = tentativas_sent.municipioV.value_counts()
fem_mun_tent = yuc[['geometry_mun','nom_mun']].merge(fem_mun_tent, right_on=fem_mun_tent.index, left_on='nom_mun', how='left')
fem_mun_tent.fillna(0,inplace=True)
fem_mun_tent.municipioV = fem_mun_tent.municipioV.astype(int)

In [ ]:
import re
def mayusculas(s):
    return ' '.join([i.strip() for i in re.findall('[A-Z][^A-Z]*', s)])

In [ ]:
def numeros(n):
    return ', '.join(n.split('.0'))

In [ ]:
def tokenize(s):
    temp = s.sub(',','')
    return ', '.join(temp)

In [ ]:
def unir_unicos(s):
    return ', '.join(set(s))

In [ ]:
def unir_resto(s):
    return ', '.join(set(s.split(' ')))

In [ ]:
feminicidios.Sentencia.value_counts()

In [ ]:
merged = fem_mun.merge(feminicidios_sent[['EdadVictima','Edad Agresor','tipos','año_recepcion','TipoRelacion','pk_perfil_agresor','municipioV']], left_on='nom_mun', right_on='municipioV', how='left')
#merged.fillna(0,inplace=True)
merged = merged.replace(np.nan, '').astype(str)
merged[merged.notnull()] = merged[merged.notnull()].astype(object)
#merged.pk_perfil_agresor = merged.pk_perfil_agresor.astype(object)
merged.drop(columns='municipioV_y',inplace=True)
merged = merged[['nom_mun','EdadVictima','Edad Agresor','tipos','año_recepcion','TipoRelacion','pk_perfil_agresor']].groupby('nom_mun').sum()

merged.TipoRelacion=merged.TipoRelacion.apply(lambda x: mayusculas(x))
merged.tipos=merged.tipos.apply(lambda x: mayusculas(x))
merged.EdadVictima=merged.EdadVictima.apply(lambda x: numeros(x))
merged['Edad Agresor']=merged['Edad Agresor'].apply(lambda x: numeros(x))
merged.pk_perfil_agresor=merged.pk_perfil_agresor.apply(lambda x: numeros(x))
merged.año_recepcion=merged.año_recepcion.apply(lambda x: numeros(x))
merged = merged.replace(',','', regex=True)


merged.tipos = merged.tipos.str.split()
merged.tipos = merged.tipos.apply(lambda x: unir_unicos(x))
merged.EdadVictima = merged.EdadVictima.apply(lambda x: unir_resto(x))
merged['Edad Agresor'] = merged['Edad Agresor'].apply(lambda x: unir_resto(x))
merged.TipoRelacion = merged.TipoRelacion.apply(lambda x: unir_resto(x))
merged.pk_perfil_agresor = merged.pk_perfil_agresor.apply(lambda x: unir_resto(x))
merged.año_recepcion = merged.año_recepcion.apply(lambda x: unir_resto(x))

test = merged.EdadVictima.apply(lambda x: x.split(', '))
merged['len'] = test.apply(lambda x: len([int(i) for i in x if i.isdigit()==True]))
test = test.apply(lambda x: sum([int(i) for i in x if i.isdigit()==True]))
merged['EdadVictima'] = round(test.values/merged.len)

test = merged['Edad Agresor'].apply(lambda x: x.split(', '))
merged['len'] = test.apply(lambda x: len([int(i) for i in x if i.isdigit()==True]))
test = test.apply(lambda x: sum([int(i) for i in x if i.isdigit()==True]))
merged['Edad Agresor'] = round(test.values/merged.len)

merged = merged.replace(np.nan, '').astype(str)
merged = merged.replace('0.0', 'Sin datos').astype(str)
#merged['EdadVictima'].values
#merged.to_csv('test.csv', encoding='utf-8-sig')

fem_mun = fem_mun.merge(merged, left_on='nom_mun', right_on=merged.index, how='left')
fem_mun.fillna('', inplace=True)

fem_mun = fem_mun.merge(pob_fem[['NOM_MUN','POBFEM']], left_on='nom_mun', right_on='NOM_MUN', how='left')

fem_mun['bins']=pd.cut(fem_mun["municipioV"],
       bins=[-1, 0.5, 1.5, 2.5,np.inf], 
       labels=['0','1','2','3 o más'])

gdf = geopandas.GeoDataFrame(fem_mun, geometry='geometry_mun')
gdf = gdf.set_index('nom_mun')
gdf = gdf.to_crs(pyproj.CRS.from_epsg(4326))

In [ ]:
merged = fem_mun_tent.merge(tentativas_sent[['EdadVictima','Edad Agresor','tipos','año_recepcion','TipoRelacion','pk_perfil_agresor','municipioV']], left_on='nom_mun', right_on='municipioV', how='left')
#merged.fillna(0,inplace=True)
merged = merged.replace(np.nan, '').astype(str)
merged[merged.notnull()] = merged[merged.notnull()].astype(object)
#merged.pk_perfil_agresor = merged.pk_perfil_agresor.astype(object)
merged.drop(columns='municipioV_y',inplace=True)
merged = merged[['nom_mun','EdadVictima','Edad Agresor','tipos','año_recepcion','TipoRelacion','pk_perfil_agresor']].groupby('nom_mun').sum()

merged.TipoRelacion=merged.TipoRelacion.apply(lambda x: mayusculas(x))
merged.tipos=merged.tipos.apply(lambda x: mayusculas(x))
merged.EdadVictima=merged.EdadVictima.apply(lambda x: numeros(x))
merged['Edad Agresor']=merged['Edad Agresor'].apply(lambda x: numeros(x))
merged.pk_perfil_agresor=merged.pk_perfil_agresor.apply(lambda x: numeros(x))
merged.año_recepcion=merged.año_recepcion.apply(lambda x: numeros(x))
merged = merged.replace(',','', regex=True)


merged.tipos = merged.tipos.str.split()
merged.tipos = merged.tipos.apply(lambda x: unir_unicos(x))
merged.EdadVictima = merged.EdadVictima.apply(lambda x: unir_resto(x))
merged['Edad Agresor'] = merged['Edad Agresor'].apply(lambda x: unir_resto(x))
merged.TipoRelacion = merged.TipoRelacion.apply(lambda x: unir_resto(x))
merged.pk_perfil_agresor = merged.pk_perfil_agresor.apply(lambda x: unir_resto(x))
merged.año_recepcion = merged.año_recepcion.apply(lambda x: unir_resto(x))

test = merged.EdadVictima.apply(lambda x: x.split(', '))
merged['len'] = test.apply(lambda x: len([int(i) for i in x if i.isdigit()==True]))
test = test.apply(lambda x: sum([int(i) for i in x if i.isdigit()==True]))
merged['EdadVictima'] = round(test.values/merged.len)

test = merged['Edad Agresor'].apply(lambda x: x.split(', '))
merged['len'] = test.apply(lambda x: len([int(i) for i in x if i.isdigit()==True]))
test = test.apply(lambda x: sum([int(i) for i in x if i.isdigit()==True]))
merged['Edad Agresor'] = round(test.values/merged.len)

merged = merged.replace(np.nan, '').astype(str)
merged = merged.replace('0.0', 'Sin datos').astype(str)
#merged['EdadVictima'].values
#merged.to_csv('test.csv', encoding='utf-8-sig')

fem_mun_tent = fem_mun_tent.merge(merged, left_on='nom_mun', right_on=merged.index, how='left')
fem_mun_tent.fillna('', inplace=True)

fem_mun_tent =fem_mun_tent.merge(pob_fem[['NOM_MUN','POBFEM']], left_on='nom_mun', right_on='NOM_MUN', how='left')

fem_mun_tent['bins']=pd.cut(fem_mun_tent["municipioV"],
       bins=[-1, 0.5, 1.5, 2.5,np.inf], 
       labels=['0','1','2','2 o más'])

gdf_tent = geopandas.GeoDataFrame(fem_mun_tent, geometry='geometry_mun')
gdf_tent = gdf_tent.set_index('nom_mun')
gdf_tent = gdf_tent.to_crs(pyproj.CRS.from_epsg(4326))

In [ ]:
# colores
colors = px.colors.qualitative.Prism[4:8]

In [ ]:
"""fig = px.choropleth_mapbox(gdf, geojson=gdf.geometry_mun, 
                    locations=gdf.index, color='bins',
                    #color_continuous_scale="Viridis",
                    color_discrete_sequence=colors,
                    labels = {'bins':'Núm. Víctimas' ,'nom_mun':'Municipio', 'municipioV':'Víctimas', 'Edad Agresor':'Edad Promedio Agresores','EdadVictima':'Edad Promedio Víctimas','tipos':'Violencias','año_recepcion':'Años','TipoRelacion':'Vínculo con agresor','pk_perfil_agresor':'pk agresor'},
                    hover_data={ 'bins':True ,"municipioV":True,'EdadVictima':True, 'Edad Agresor':True , 'tipos':True,'año_recepcion':True,'TipoRelacion':True,'pk_perfil_agresor':False},
                    title='Mapa 3. Feminicidios Registrados por Municipio',
                    #color_continuous_midpoint=np.average(gdf['municipioV'], weights=gdf['POBFEM'].astype(int)),
                    center={"lat": 20.400417, "lon": -89.134857},
                    mapbox_style="carto-positron", 
                    zoom=7.5,
                    #opacity=0.5,
                    )
fig.update_geos(fitbounds="locations", visible=False)

#fig.update_layout(xaxis=dict(domain=[0, 0.5]), yaxis=dict(domain=[0.25, 0.75]))

fig.add_annotation(
    # The arrow head will be 25% along the x axis, starting from the left
    x=0,
    # The arrow head will be 40% along the y axis, starting from the bottom
    y=0.98,
    text="<b>Temporalidad:</b> {}-{}<br><b>Total de casos:</b> {}".format(min(feminicidios.año_recepcion),max(feminicidios.año_recepcion),(gdf.municipioV.sum())),
    showarrow=False,
    bordercolor="black",
    bgcolor="white",
    borderwidth=1.5,
    opacity=0.8
)

fig.update(layout = dict(title=dict(x=0.5)))
fig.update_layout(
    margin={"r":0,"t":30,"l":10,"b":10},
    coloraxis_colorbar={
        'title':'Log10'})
fig.update_layout(title_y=1, title_x=0)
fig.update_layout(coloraxis_showscale=True)
#pio.write_html(fig, file='../docs/figs/indicador_municipal_mapa_feminicidios_escalado.html', include_plotlyjs='cdn', full_html=False)
fig.show()"""

In [ ]:
# fem y tent
cols_dd = ['Feminicidios','Tentativas']
# we need to add this to select which trace 
# is going to be visible
visible = np.array(cols_dd)

poly_json = json.loads(gdf.geometry_mun.to_json())

# define traces and buttons at once
traces = []
buttons = []

# Agregar mapa general
traces.append(go.Choroplethmapbox(
        geojson=poly_json,
        locations=gdf.index, # Spatial coordinates
        z=gdf['municipioV'], # Data to be color-coded
        colorscale=[[0, colors[0]],[0.25, colors[0]],
        [0.25, colors[1]],[0.5, colors[1]],
        [0.5, colors[2]],[0.75, colors[2]],
        [0.75, colors[3]], [1, colors[3]],],
        zmin=0, zmax=3,
        colorbar=dict(
        title="Núm. Víctimas<br>",
        titleside="top",
        tickmode="array",
        tickvals=[0.375, 1.125, 1.875, 2.625],
        ticktext=["0", "1", "2", "3 o más"],
        ticks="outside"
        ),
        #colorbar_title='Casos',
        name='Feminicidios',
        #hoverlabel={'Tasa 1000 Total':False},
        #hovertemplate = "Mun_x: %{z}<br>Ola: %{customdata[0]}<br>Fin: %{customdata[-1]}",
        #text = gdf['text_total'], # hover text
        text = gdf.index, # hover text
        customdata=np.stack([gdf['TipoRelacion'], gdf['EdadVictima'], gdf['Edad Agresor'], gdf['tipos'], gdf['año_recepcion'], gdf['TipoRelacion']], axis=-1),
        hovertemplate='<b>%{text}</b>'+ '<br>' +'Número de Víctimas: ' + '%{z}' +  '<br>' +'Promedio Edad Víctimas: ' 
        + '%{customdata[1]}' +'<br>Promedio Edad Agresores: ' + '%{customdata[2]}'+'<br>Tipos de violencia: ' + '%{customdata[3]}'
        +'<br>Año de recepción del caso: ' + '%{customdata[4]}' + '<br>Vínculo con el agresor: ' + '%{customdata[5]}',
        visible= True
        ))

buttons.append(dict(label='Feminicidios',
                    method="update",
                    args=[{"visible":list(visible=='Feminicidios')},
                            {"title":f"Mapa 2. Feminicidios y Tentativas Registrados por Municipio"}]))

poly_json = json.loads(gdf_tent.geometry_mun.to_json())
# Agregar mapa general
traces.append(go.Choroplethmapbox(
        geojson=poly_json,
        locations=gdf_tent.index, # Spatial coordinates
        z=gdf_tent['municipioV'], # Data to be color-coded
        colorscale=[[0, colors[0]],[0.25, colors[0]],
        [0.25, colors[1]],[0.5, colors[1]],
        [0.5, colors[2]],[0.75, colors[2]],
        [0.75, colors[3]], [1, colors[3]],],
        zmin=0, zmax=3,
        colorbar=dict(
        title="Núm. Víctimas<br>",
        titleside="top",
        tickmode="array",
        tickvals=[0.375, 1.125, 1.875, 2.625],
        ticktext=["0", "1", "2", "3 o más"],
        ticks="outside"
        ),
        #colorbar_title='Casos',
        name='Tentativas',
        #hoverlabel={'Tasa 1000 Total':False},
        #hovertemplate = "Mun_x: %{z}<br>Ola: %{customdata[0]}<br>Fin: %{customdata[-1]}",
        #text = gdf['text_total'], # hover text
        text = gdf_tent.index, # hover text
        customdata=np.stack([gdf_tent['TipoRelacion'], gdf_tent['EdadVictima'], gdf_tent['Edad Agresor'], gdf_tent['tipos'],gdf_tent['año_recepcion'], gdf_tent['TipoRelacion']], axis=-1),
        hovertemplate='<b>%{text}</b>'+ '<br>' +'Número de Víctimas: ' + '%{z}' +  '<br>' +'Promedio Edad Víctimas: ' 
        + '%{customdata[1]}' +'<br>Promedio Edad Agresores: ' + '%{customdata[2]}'+'<br>Tipos de violencia: ' + '%{customdata[3]}'
        +'<br>Año de recepción del caso: ' + '%{customdata[4]}' + '<br>Vínculo con el agresor: ' + '%{customdata[5]}',
        visible= False,
        ))

buttons.append(dict(label='Tentativas',
                    method="update",
                    args=[{"visible":list(visible=='Tentativas')},
                            {"title":f"Mapa 2. Feminicidios y Tentativas Registrados por Municipio"}]))


updatemenus = [{"active":0,
                "buttons":buttons,
               }]


# Show figure
fig = go.Figure(data=traces,
                layout=dict(updatemenus=updatemenus))
# This is in order to get the first title displayed correctly
#first_title = cols_dd[0]
fig.update_layout(title=f"Mapa 2. Feminicidios y Tentativas Registrados por Municipio",)
fig.update_geos(fitbounds="locations", visible=False, )
fig.add_annotation(
    # The arrow head will be 25% along the x axis, starting from the left
    x=0,
    # The arrow head will be 40% along the y axis, starting from the bottom
    y=0.98,
    text="<b>Temporalidad:</b> {}-{}<br><b>Total de casos:</b> {}".format(min(feminicidios.año_recepcion),max(feminicidios.año_recepcion),(len(feminicidios))),
    showarrow=False,
    bordercolor="black",
    bgcolor="white",
    borderwidth=1.5,
    opacity=0.8
)

fig.update_layout(mapbox_style="carto-positron",
                  mapbox_zoom=7.5, mapbox_center = {"lat": 20.400417, "lon": -89.134857})
pio.write_html(fig, file='../docs/figs/indicador_municipal_mapa_fem_tent.html', include_plotlyjs='cdn', full_html=False)
fig.show()

In [ ]:
fig = px.treemap(gdf,
                 path=[px.Constant("Yucatán"), 'NOM_MUN'],
                 values='municipioV',
                 hover_data={"municipioV":True,'EdadVictima':True,'tipos':True,'año_recepcion':True,'TipoRelacion':True,'pk_perfil_agresor':True, 'Edad Agresor':True},
                 labels = {'nom_mun':'Municipio', 'municipioV':'Víctimas','EdadVictima':'Edad Promedio Víctima','Edad Agresor':'Edad Promedio Agresor','tipos':'Violencias','año_recepcion':'Años','TipoRelacion':'Relación','pk_perfil_agresor':'pk agresor'},
                 title='Gráfica 2. Feminicidios Registrados por Municipio {}-{}'.format(min(feminicidios.año_recepcion),max(feminicidios.año_recepcion)),
                 #text='ola'
                 )

#fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
print("plotly express hovertemplate:", fig.data[0].hovertemplate)
fig.update_traces(hovertemplate='Víctimas=%{customdata[0]}<br>Edad Promedio Víctima=%{customdata[1]}<br>Edad Promedio Agresor=%{customdata[6]}<br>Violencias=%{customdata[2]}<br>Años=%{customdata[3]}')
fig.update_traces(textinfo="label+value+percent root",)
pio.write_html(fig, file='../docs/figs/indicador_municipal_treemap_feminicidios.html', include_plotlyjs='cdn', full_html=False)
fig.show()

In [ ]:
casos_mun = victimas_2020.groupby('municipiohechos')['fk_euv'].count()
casos_mun = yuc[['geometry_mun','nom_mun']].merge(casos_mun, right_on=casos_mun.index, left_on='nom_mun', how='left')
casos_mun.fillna(0,inplace=True)
casos_mun = pd.DataFrame(casos_mun.merge(pob_fem,left_on='nom_mun',right_on='NOM_MUN',how='left'))
casos_mun[['fk_euv','POBFEM']] = casos_mun[['fk_euv','POBFEM']].astype(int)

In [ ]:
mun_violencias = victimas_2020.groupby('municipiohechos')[['Psicológica','Física','Económica','Sexual','Patrimonial','Feminicida','Otro']].sum()
mun_violencias = yuc[['geometry_mun','nom_mun']].merge(mun_violencias, right_on=mun_violencias.index, left_on='nom_mun', how='left')
#mun_violencias = mun_violencias.merge(yuc[['geometry_mun','nom_mun']], left_on=mun_violencias.index, right_on='nom_mun', how='right')
#for violencia in ['Psicológica','Física','Económica','Sexual','Patrimonial','Feminicida','Otro']:
    #total = mun_violencias[violencia].sum()
    #mun_violencias['{}_rel'.format(violencia)] = round(mun_violencias[violencia]/total*100,2)
mun_violencias.fillna(0, inplace=True)

In [ ]:
mun_violencias = pd.DataFrame(mun_violencias.merge(pob_fem,left_on='nom_mun',right_on='NOM_MUN',how='left'))
for violencia in ['Psicológica','Física','Económica','Sexual','Patrimonial','Feminicida','Otro']:
    mun_violencias['{} Tasa 1000'.format(violencia)] = round(mun_violencias[violencia]/mun_violencias['POBFEM'].astype(int)*1000,2)
    #mun_violencias['{}_text'.format(violencia)] = mun_violencias.apply(lambda row: f"Casos: {int(row[violencia])}<br>POBFEM: {int(row['POBFEM'])}", axis=1)
    mun_violencias['{}_casos'.format(violencia)] = mun_violencias.apply(lambda row: int(row[violencia]), axis=1)

mun_violencias = mun_violencias.merge(casos_mun[['fk_euv','nom_mun']], left_on=mun_violencias.index, right_on=casos_mun.index, how='left')
mun_violencias[['fk_euv','POBFEM']] = mun_violencias[['fk_euv','POBFEM']].astype(int)
mun_violencias['Tasa 1000 Total'] =round(mun_violencias.fk_euv/mun_violencias.POBFEM*1000,2)
mun_violencias.drop(['key_0','nom_mun_y','NOM_MUN','MUN'],axis='columns', inplace=True)
#mun_violencias['text_total'] = mun_violencias.apply(lambda row: f"Casos Totales: {int(row['fk_euv'])}<br>POBFEM: {row['POBFEM']}", axis=1)

gdf = geopandas.GeoDataFrame(mun_violencias, geometry='geometry_mun')
gdf = gdf.set_index('nom_mun_x')
gdf = gdf.to_crs(pyproj.CRS.from_epsg(4326))


In [ ]:
cols_dd = ['Tasa 1000 Total','Psicológica','Física','Económica','Sexual','Patrimonial','Feminicida','Otro',]
# we need to add this to select which trace 
# is going to be visible
visible = np.array(cols_dd)

poly_json = json.loads(gdf.geometry_mun.to_json())

# define traces and buttons at once
traces = []
buttons = []

# Agregar mapa general
traces.append(go.Choroplethmapbox(
        geojson=poly_json,
        locations=gdf.index, # Spatial coordinates
        z=gdf['Tasa 1000 Total'], # Data to be color-coded
        colorbar_title='Tasa',
        name='Tasa 2020',
        #hoverlabel={'Tasa 1000 Total':False},
        #hovertemplate = "Mun_x: %{z}<br>Ola: %{customdata[0]}<br>Fin: %{customdata[-1]}",
        #text = gdf['text_total'], # hover text
        text = gdf.index, # hover text
        customdata=np.stack([ gdf['POBFEM'], gdf['fk_euv']], axis=-1),
        hovertemplate='<b>%{text}</b>'+ '<br>' +'Tasa: ' + '%{z}' +  '<br>' +'Casos Totales: ' + '%{customdata[1]}' +'<br>Población Femenina: ' + '%{customdata[0]}',
        visible= True if 'Tasa 1000 Total'==cols_dd[0] else False,
        ))

buttons.append(dict(label='Tasa 1000 Total',
                    method="update",
                    args=[{"visible":list(visible=='Tasa 1000 Total')},
                            {"title":f"Mapa 1. Tasa de casos de violencia por cada 1000 mujeres"}]))

# Mapas de tipos de violencia
for value in cols_dd[1:]:
    traces.append(go.Choroplethmapbox(
        geojson=poly_json,
        locations=gdf.index, # Spatial coordinates
        z=gdf['{} Tasa 1000'.format(value)], # Data to be color-coded
        colorbar_title='Tasa',
        name='Tasa de violencia<br>'+value,
        #hoverinfo='{}+{} Tasa 1000'.format(value,value),
        #hovertemplate = "Mun: %{x}".format(gdf.index),
        #text = gdf['{}_text'.format(value)], # hover text
        visible= True if value==cols_dd[0] else False,
        marker_line_width=1,
        colorscale="Viridis",
        text = gdf.index, # hover text
        customdata=np.stack([ gdf['POBFEM'], gdf['{}_casos'.format(value)]], axis=-1),
        hovertemplate='<b>%{text}</b>'+ '<br>' +'Tasa: ' + '%{z}' +  '<br>' +'Casos: ' + '%{customdata[1]}' +'<br>Población Femenina: ' + '%{customdata[0]}',
        #visible= False,
        ))
                                    

    buttons.append(dict(label=value,
                        method="update",
                        args=[{"visible":list(visible==value)},
                              {"title":f"Mapa 1. Tasa de casos de violencia {value} por cada 1000 mujeres"}]))


updatemenus = [{"active":0,
                "buttons":buttons,
               }]


# Show figure
fig = go.Figure(data=traces,
                layout=dict(updatemenus=updatemenus))
# This is in order to get the first title displayed correctly
first_title = cols_dd[0]
fig.update_layout(title=f"Mapa 1. Tasa de casos de violencia por cada 1000 mujeres",)
fig.update_geos(fitbounds="locations", visible=False, )
fig.add_annotation(
    # The arrow head will be 25% along the x axis, starting from the left
    x=0,
    # The arrow head will be 40% along the y axis, starting from the bottom
    y=0.98,
    text="<b>Temporalidad:</b> {}<br><b>Total de casos:</b> {}".format(min(victimas_2020.año_recepcion),len(victimas_2020)),
    showarrow=False,
    bordercolor="black",
    bgcolor="white",
    borderwidth=1.5,
    opacity=0.8
)

fig.update_layout(mapbox_style="carto-positron",
                  mapbox_zoom=7.5, mapbox_center = {"lat": 20.400417, "lon": -89.134857})
pio.write_html(fig, file='../docs/figs/indicador_municipal_mapa_violencias.html', include_plotlyjs='cdn', full_html=False)
fig.show()

In [ ]:
fem_agrav = feminicidios[feminicidios.Sentencia.isin(['FEMINICIDIO AGRAVADO','FEMINICIDIO'])].index
feminicidios.loc[fem_agrav,'Sentencia']='FEMINICIDIO'
fem_agrav = feminicidios[feminicidios.Sentencia.isin(['FEMINICIDIO AGRAVADO EN GRADO DE TENTATIVA','FEMINICIDIO EN GRADO DE TENTATIVA'])].index
feminicidios.loc[fem_agrav,'Sentencia']='TENTATIVA DE FEMINICIDIO'

In [ ]:
sentencias = feminicidios.Sentencia.value_counts()

In [ ]:
#
desc = victimas[victimas['Portaba Arma']=='Se desconoce'].index
victimas.loc[desc,'Portaba Arma']='No'
armas = victimas['Portaba Arma'].value_counts()


In [ ]:
# Use `hole` to create a donut-like pie chart
fig = go.Figure(data=[go.Pie(labels=sentencias.index, values=sentencias.values, hole=.3 )])
fig.update_layout(title='Gráfica 3. Porcentaje de Presuntos Feminicidios con Sentencia')
fig.update_traces(textinfo="label+value+percent",showlegend=False, textposition='outside')
pio.write_html(fig, file='../docs/figs/indicador_municipal_donut_sentencias.html', include_plotlyjs='cdn', full_html=False)
#fig.update_traces(hovertemplate='Servicio=%{label}<br>Número de servicios=%{value}<br>Origen=%{parent}<br><extra></extra>')
fig.show()

In [ ]:
fig = go.Figure(data=[go.Pie(labels=armas.index, values=armas.values, hole=.3 )])
fig.update_layout(title='Gráfica 4. Porcentaje de agresores armados')
fig.update_traces(textinfo="label+value+percent",showlegend=False,)
pio.write_html(fig, file='../docs/figs/indicador_municipal_donut_armas.html', include_plotlyjs='cdn', full_html=False)

fig.show()

In [ ]:
armas_mun = victimas[victimas['Portaba Arma']=='Sí']

In [ ]:
armas_mun = pd.DataFrame(armas_mun.groupby('municipiohechos')['Portaba Arma'].count())
armas_mun = armas_mun.merge(yuc[['nom_mun','geometry_mun']], left_on=armas_mun.index, right_on='nom_mun', how='right')
armas_mun.fillna(0, inplace=True)
armas_mun['Portaba Arma'] = armas_mun['Portaba Arma'].astype(int)

In [ ]:
fig = px.treemap(armas_mun,
                 path=[px.Constant("Yucatán"), 'nom_mun'],
                 values='Portaba Arma',
                 #hover_data={"municipioV":True,'EdadVictima':True,'tipos':True,'año_recepcion':True,'TipoRelacion':True,'pk_perfil_agresor':True},
                 #labels = {'nom_mun':'Municipio', 'municipioV':'Víctimas','EdadVictima':'Edades','tipos':'Violencias','año_recepcion':'Años','TipoRelacion':'Relación','pk_perfil_agresor':'pk agresor'},
                 title='Gráfica 5. Municipios en los que los agresores han portado armas',
                 )

#fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
#print("plotly express hovertemplate:", fig.data[0].hovertemplate)
fig.update_traces(hovertemplate='Municipio=%{label}<br>Portaban Arma=%{value}<br><extra></extra>')
fig.update_traces(textinfo="label+value+percent root",)
pio.write_html(fig, file='../docs/figs/indicador_municipal_treemap_arma.html', include_plotlyjs='cdn', full_html=False)
fig.show()

In [ ]:
gdf = geopandas.GeoDataFrame(armas_mun, geometry='geometry_mun')
gdf = gdf.set_index('nom_mun')
gdf = gdf.to_crs(pyproj.CRS.from_epsg(4326))

In [ ]:
"""gdf['log']=np.log10(gdf["Portaba Arma"])
inf = gdf[gdf.log==-np.inf].index
gdf.loc[inf,'log'] = -1"""

In [ ]:
gdf['cat']=pd.cut(gdf['Portaba Arma'],
       bins=[-1, 0.9, 10.9,20.9,35.9,np.inf], 
       labels=['0','1-10','11-20','21-35','36 o más'])

In [ ]:
gdf['cat'].values.unique()

In [ ]:
gdf = gdf.sort_values('Portaba Arma')

In [ ]:
# colores
colors = px.colors.qualitative.Prism[3:8]

In [ ]:
fig = px.choropleth_mapbox(gdf, geojson=gdf.geometry_mun, 
                    locations=gdf.index, color='cat',
                    color_discrete_sequence=colors,
                    labels = {'nom_mun':'Municipio', 'Portaba Arma':'Agresores Armados','cat':'Agresores<br>Armados'},
                    hover_data={'Portaba Arma':True},
                    #title='Mapa 2. Feminicidios Registrados por Municipio',
                    #color_continuous_midpoint=np.average(gdf['municipioV'], weights=gdf['POBFEM'].astype(int)),
                    center={"lat": 20.400417, "lon": -89.134857},
                    mapbox_style="carto-positron", 
                    zoom=7.5,
                    #opacity=0.5,
                    title='Mapa 3. Agresores Armados por Municipio',
                    )
fig.update_geos(fitbounds="locations", visible=False)

#fig.update_layout(xaxis=dict(domain=[0, 0.5]), yaxis=dict(domain=[0.25, 0.75]))

fig.add_annotation(
    # The arrow head will be 25% along the x axis, starting from the left
    x=0,
    # The arrow head will be 40% along the y axis, starting from the bottom
    y=0.98,
    text="<b>Temporalidad:</b> {}-{}<br><b>Total de casos:</b> {}".format(min(victimas.año_recepcion),max(victimas.año_recepcion),len(victimas)),
    showarrow=False,
    bordercolor="black",
    bgcolor="white",
    borderwidth=1.5,
    opacity=0.8
)

fig.update(layout = dict(title=dict(x=0.5)))
fig.update_layout(title_y=1, title_x=0)
print(fig.data[0].hovertemplate)
fig.update_traces(hovertemplate='Municipio=%{location}<br>Agresores Armados=%{customdata[0]}<extra></extra>')
pio.write_html(fig, file='../docs/figs/indicador_municipal_mapa_arma.html', include_plotlyjs='cdn', full_html=False)
fig.show()

In [ ]:
edad_agresores = victimas[(victimas['Edad Agresor']>0)&(victimas['Edad Agresor']<99)]

In [ ]:
fig = px.histogram(edad_agresores, x="Edad Agresor", color="Tipo de vínculo con victima", marginal="box",opacity=0.8,nbins=20,
            labels = {'Edad Agresor': 'Grupo Etario',},
            title = 'Gráfica 6. Distribución de agresores por vínculo con la víctima y edad'
            ) # can be `box`, `violin`
pio.write_html(fig, file='../docs/figs/indicador_municipal_dist_edad_agresor.html', include_plotlyjs='cdn', full_html=False)
fig.show()

In [ ]:
fig = px.histogram(victimas, x="Edad", color="Esta Embarazada", marginal="box",opacity=0.8,
            labels = {'Edad': 'Grupo Etario'},
            title = 'Gráfica 7. Distribución de casos por embarazo y edad',
            nbins=20
            ) # can be `box`, `violin`
pio.write_html(fig, file='../docs/figs/indicador_municipal_dist_victimas.html', include_plotlyjs='cdn', full_html=False)
fig.show()

In [ ]:
"""fig = px.histogram(victimas, x="Edad", marginal="box",opacity=0.8,
            labels = {'Edad': 'Grupo Etario'},
            title = 'Gráfica #. Distribución de víctimas por grupo etario',
            nbins=20
            ) # can be `box`, `violin`
#pio.write_html(fig, file='../docs/figs/indicador_municipal_dist_victimas_etario.html', include_plotlyjs='cdn', full_html=False)
fig.show()"""

In [ ]:
mun_edad = pd.DataFrame(victimas.groupby('municipiohechos')['Edad'].mean().round().astype(int)) # ya que es una distribucion normal, se puede usar la media como agregacion
mun_edad = mun_edad.rename(columns={'Edad':'Edad Promedio'})

In [ ]:
mun_edad = mun_edad.merge(yuc[['nom_mun','geometry_mun']], left_on=mun_edad.index, right_on='nom_mun', how='right')
mun_edad.fillna(0, inplace=True)
gdf = geopandas.GeoDataFrame(mun_edad, geometry='geometry_mun')
gdf = gdf.set_index('nom_mun')
gdf = gdf.to_crs(pyproj.CRS.from_epsg(4326))

In [ ]:
fig = px.choropleth_mapbox(gdf, geojson=gdf.geometry_mun, 
                    locations=gdf.index, color="Edad Promedio",
                    color_continuous_scale="Viridis",
                    labels={'nom_mun': 'Municipio'},
                    #labels = {'nom_mun':'Municipio', 'municipioV':'Víctimas','EdadVictima':'Edades','tipos':'Violencias','año_recepcion':'Años','TipoRelacion':'Relación','pk_perfil_agresor':'pk agresor'},
                    #hover_data={"municipioV":True,'EdadVictima':True,'tipos':True,'año_recepcion':True,'TipoRelacion':True,'pk_perfil_agresor':True,'Hubo Feminicidio':False },
                    title='Mapa 4. Edad Promedio de las Víctimas por Municipio',
                    #color_continuous_midpoint=np.average(victimas['Edad']),
                    center={"lat": 20.400417, "lon": -89.134857},
                    mapbox_style="carto-positron", 
                    zoom=7.5,
                    #opacity=0.5,
                    )
fig.update_geos(fitbounds="locations", visible=False)

#fig.update_layout(xaxis=dict(domain=[0, 0.5]), yaxis=dict(domain=[0.25, 0.75]))

fig.add_annotation(
    # The arrow head will be 25% along the x axis, starting from the left
    x=0,
    # The arrow head will be 40% along the y axis, starting from the bottom
    y=0.98,
    text="<b>Temporalidad:</b> {}-{}<br><b>Total de casos:</b> {}".format(min(victimas.año_recepcion),max(victimas.año_recepcion),len(victimas)),
    showarrow=False,
    bordercolor="black",
    bgcolor="white",
    borderwidth=1.5,
    opacity=0.8
)

fig.update(layout = dict(title=dict(x=0.5)))
fig.update_layout(
    margin={"r":0,"t":30,"l":10,"b":10},
    coloraxis_colorbar={
        'title':'Edad'})
fig.update_layout(title_y=1, title_x=0)
fig.update_layout(coloraxis_showscale=True)
pio.write_html(fig, file='../docs/figs/indicador_municipal_mapa_edad_promedio.html', include_plotlyjs='cdn', full_html=False)
fig.show()

In [ ]:
servicios = pd.DataFrame(servicios[servicios.dependenciaquebrindoservicio.isna()==False])

In [ ]:
semujeres = servicios[servicios.dependenciaquebrindoservicio=='SECRETARÍA DE LAS MUJERES'].index
prodenay = servicios[servicios.dependenciaquebrindoservicio=='PROCURADURÍA DE LA DEFENSA DEL MENOR Y LA FAMILIA'].index
ssp = servicios[servicios.dependenciaquebrindoservicio=='SECRETARÍA DE SEGURIDAD PÚBLICA'].index
ceeav = servicios[servicios.dependenciaquebrindoservicio=='COMISIÓN EJECUTIVA ESTATAL DE ATENCIÓN A VÍCTIMAS'].index
segey = servicios[servicios.dependenciaquebrindoservicio== 'SECRETARÍA DE EDUCACIÓN DEL GOBIERNO DEL ESTADO DE YUCATÁN'].index
servicios.loc[semujeres, 'dependenciaquebrindoservicio'] = 'SEMUJERES'
servicios.loc[prodenay, 'dependenciaquebrindoservicio'] = 'PRODENNAY'
servicios.loc[ssp, 'dependenciaquebrindoservicio'] = 'SSP'
servicios.loc[ceeav, 'dependenciaquebrindoservicio'] = 'CEEAV'
servicios.loc[segey, 'dependenciaquebrindoservicio'] = 'SEGEY'

In [ ]:
selecciones = servicios[servicios.tiposervicio=='Seleccione'].index
servicios.loc[selecciones, 'tiposervicio'] = 'No identificado'

In [ ]:
servicios_gropby = pd.DataFrame(servicios.groupby(['dependenciaquebrindoservicio','tiposervicio','serviciodetalle'])['numeroservicios'].count()).reset_index()

In [ ]:
fig = px.sunburst(servicios_gropby, path=['dependenciaquebrindoservicio','tiposervicio','serviciodetalle'], values='numeroservicios', color='tiposervicio',
                #labels={'labels':'Servicio Detalle'}, 
                #hover_data={'id':False},
                title='Gráfica 8. Distribución de los servicios por tipo e instancia capturista'
                )
print("plotly express hovertemplate:", fig.data[0].hovertemplate)
fig.update_traces(hovertemplate='Servicio Detalle=%{label}<br>Número de Servicios=%{value}<br>Pertenencia=%{parent}<br><extra></extra>')
fig.update_traces(textinfo="label+value+percent entry",)
pio.write_html(fig, file='../docs/figs/indicador_municipal_sunburst_servicios.html', include_plotlyjs='cdn', full_html=False)
fig.show()

In [ ]:
"""fig = px.treemap(servicios_gropby,
                 #path=[, 'nom_mun'],
                 path=[px.Constant("Yucatán"), 'dependenciaquebrindoservicio','tiposervicio','serviciodetalle'],
                 values='numeroservicios',
                 title='Gráfica #. Distribución de los servicios por tipo e instancia capturista',
                 )

#fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
print("plotly express hovertemplate:", fig.data[0].hovertemplate)
#fig.update_traces(hovertemplate='Servicio=%{label}<br>Número de servicios=%{value}<br>Origen=%{parent}<br><extra></extra>')
fig.update_traces(textinfo="label+value+percent root",)
#pio.write_html(fig, file='../docs/figs/indicador_municipal_treemap_servicios.html', include_plotlyjs='cdn', full_html=False)
fig.show()"""

In [ ]:
fechas_groupby = pd.DataFrame(victimas.groupby('fecha_hechos')['pk_caso'].count())
fechas_groupby = fechas_groupby.rename(columns={'pk_caso':'Casos registrados'})

In [ ]:
fechas_groupby['var'] = round(fechas_groupby['Casos registrados'].pct_change()*100,2)

In [ ]:
fechas_groupby = pd.DataFrame(fechas_groupby[fechas_groupby.index >= '1990-01-01'])

In [ ]:
semanas_groupby = pd.DataFrame(fechas_groupby.reset_index().groupby(pd.Grouper(freq='W', key='fecha_hechos'))['Casos registrados'].sum()).reset_index()
semanas_groupby['var'] = round(semanas_groupby['Casos registrados'].pct_change()*100,2)
inf = semanas_groupby[semanas_groupby['var'].isin([np.inf, -np.inf])].index
#semanas_groupby.loc[inf,'var']=semanas_groupby.loc[inf,'Casos registrados']*100
semanas_groupby.loc[inf,'var']=100
semanas_groupby

In [ ]:
# agregado para quitar aquellas en 0
#semanas_groupby=semanas_groupby[~semanas_groupby['var'].isin([np.nan, np.inf, -np.inf])]
inf = semanas_groupby[semanas_groupby['var'].isin([np.nan])].index
#semanas_groupby.loc[inf,'var']=semanas_groupby.loc[inf,'Casos registrados']*100
semanas_groupby.loc[inf,'var']=0
semanas_groupby

In [ ]:
fig = px.line(semanas_groupby, x=semanas_groupby.fecha_hechos, y="Casos registrados", title='Grafica 9. Variación de casos de violencia hacia las mujeres',
                hover_data={"var": True}, labels={'fecha_hechos':'Fecha Hechos'},markers=True,
                range_x=['2015-01-01',max(semanas_groupby.fecha_hechos)],
                #range_y=[0,500]
                )
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            #dict(step="all")
        ])
    )
)

print("plotly express hovertemplate:", fig.data[0].hovertemplate)
fig.update_traces(hovertemplate='Fecha=%{x}<br>Casos registrados=%{y}<br>Variación=%{customdata[0]}%<extra></extra>')
#fig.update_yaxes(type="log")
pio.write_html(fig, file='../docs/figs/indicador_municipal_timeline_variaciones.html', include_plotlyjs='cdn', full_html=False)
fig.show()

In [ ]:
"""fig = px.line(fechas_groupby, x=fechas_groupby.index, y="Casos registrados", title='Grafica #. Variación de casos de violencia hacia las mujeres')
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
#pio.write_html(fig, file='../docs/figs/indicador_municipal_sunburst_servicios.html', include_plotlyjs='cdn', full_html=False)
fig.show()"""

In [ ]:
"""fig = px.line(semanas_groupby, x=semanas_groupby.fecha_recepcion, y="var", title='Grafica #. Variación porcentual de casos de violencia hacia las mujeres')
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.update_layout(yaxis_ticksuffix = '%')
#pio.write_html(fig, file='../docs/figs/indicador_municipal_sunburst_servicios.html', include_plotlyjs='cdn', full_html=False)
fig.show()"""

In [ ]:
"""fig = px.line(fechas_groupby, x=fechas_groupby.index, y="var", title='Grafica #. Variación porcentual de casos de violencia hacia las mujeres')
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.update_layout(yaxis_ticksuffix = '%')
#pio.write_html(fig, file='../docs/figs/indicador_municipal_sunburst_servicios.html', include_plotlyjs='cdn', full_html=False)
fig.show()"""

In [ ]:
#fechas_groupby["Color"] = np.where(fechas_groupby["var"]<0, 'Disminución', 'Aumento')
#semanas_groupby["Color"] = np.where(semanas_groupby["var"]<0, 'Disminución', 'Aumento')

In [ ]:
"""fig = px.bar(fechas_groupby, x=fechas_groupby.index, y="var", title='Grafica #. Variación porcentual de casos de violencia hacia las mujeres', color='Color')
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.update_layout(yaxis_ticksuffix = '%')
#fig.update_xaxes(showgrid=True, ticklabelmode="period", dtick="M1", tickformat="%b\n%Y")
#pio.write_html(fig, file='../docs/figs/indicador_municipal_timeline_variaciones.html', include_plotlyjs='cdn', full_html=False)
fig.show()"""

In [ ]:
"""# semanal group
fig = px.histogram(semanas_groupby, x=semanas_groupby.fecha_recepcion, y="var", title='Grafica #. Variación porcentual de casos de violencia hacia las mujeres', color='Color', )
fig.update_traces(xbins_size="W")

fig.update_layout(bargap=0.1)
#fig.add_trace(go.Scatter(mode="markers", x=fechas_groupby.index, y=fechas_groupby.var, name="daily"))

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)

#fig.update_xaxes(showgrid=True, ticklabelmode="period", dtick="M1", tickformat="%b\n%Y")
fig.update_xaxes(showgrid=True)
fig.update_layout(yaxis_ticksuffix = '%')
#pio.write_html(fig, file='../docs/figs/indicador_municipal_timeline_histograma_semanal.html', include_plotlyjs='cdn', full_html=False)
fig.show() """

In [ ]:
"""# DIARIO
fig = px.histogram(fechas_groupby, x=fechas_groupby.index, y="var", title='Grafica #. Variación porcentual de casos de violencia hacia las mujeres', color='Color', )
fig.update_traces(xbins_size="D")

fig.update_layout(bargap=0.1)
#fig.add_trace(go.Scatter(mode="markers", x=fechas_groupby.index, y=fechas_groupby.var, name="daily"))

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)

#fig.update_xaxes(showgrid=True, ticklabelmode="period", dtick="M1", tickformat="%b\n%Y")
fig.update_xaxes(showgrid=True)
fig.update_layout(yaxis_ticksuffix = '%')
#pio.write_html(fig, file='../docs/figs/indicador_municipal_timeline_histograma.html', include_plotlyjs='cdn', full_html=False)
fig.show()  """

In [ ]:
casos_2020 = victimas_2020.groupby('municipiohechos')['fk_euv'].count()
casos_2020 = yuc[['geometry_mun','nom_mun']].merge(casos_2020, right_on=casos_2020.index, left_on='nom_mun', how='left')
casos_2020.fillna(0,inplace=True)
casos_2020['fk_euv']=casos_2020['fk_euv'].astype(int)
casos_2020 = casos_2020.rename(columns={'fk_euv':'casos_2020'})

In [ ]:
casos_2019 = victimas_2019.groupby('municipiohechos')['fk_euv'].count()
casos_2019 = yuc[['geometry_mun','nom_mun']].merge(casos_2019, right_on=casos_2019.index, left_on='nom_mun', how='left')
casos_2019.fillna(0,inplace=True)
casos_2019['fk_euv']=casos_2019['fk_euv'].astype(int)
casos_2019 = casos_2019.rename(columns={'fk_euv':'casos_2019'})

In [ ]:
dif_casos = casos_2020.merge(casos_2019[['nom_mun','casos_2019']], left_on='nom_mun', right_on='nom_mun', how='left')

In [ ]:
dif_casos

In [ ]:
var = round(dif_casos[['casos_2019','casos_2020']].pct_change(axis='columns')*100,2)
dif_casos = dif_casos.merge(var['casos_2020'], left_on=dif_casos.index, right_on=var.index, how='left')
dif_casos.drop(columns=['key_0'], inplace=True)

In [ ]:
dif_casos = dif_casos.rename(columns={'casos_2020_y':'Variación'})

In [ ]:
dif_casos

In [ ]:
total_casos=dif_casos.casos_2020_x.sum()+dif_casos.casos_2019.sum()
total_casos

In [ ]:
gdf = geopandas.GeoDataFrame(dif_casos, geometry='geometry_mun')
gdf = gdf.set_index('nom_mun')
gdf = gdf.to_crs(pyproj.CRS.from_epsg(4326))

In [ ]:
inf = gdf[gdf.Variación.isin([np.inf])].index
gdf.loc[inf,'Variación']=gdf.loc[inf,'casos_2020_x']*100

In [ ]:
gdf['total'] = gdf['casos_2020_x'] + gdf['casos_2019']

In [ ]:
gdf

In [ ]:
fig = px.choropleth_mapbox(gdf, geojson=gdf.geometry_mun, 
                    locations=gdf.index, color="Variación",
                    color_continuous_scale="RdBu_r",
                    labels={'nom_mun': 'Municipio',"casos_2020_x":'Casos 2020','casos_2019':'Casos 2019' },
                    #color_continuous_midpoint=np.average(gdf['Variación']),
                    #color_continuous_midpoint=0,
                    #labels = {'nom_mun':'Municipio', 'municipioV':'Víctimas','EdadVictima':'Edades','tipos':'Violencias','año_recepcion':'Años','TipoRelacion':'Relación','pk_perfil_agresor':'pk agresor'},
                    hover_data={"casos_2020_x":True,'casos_2019':True },
                    title='Mapa 5. Variación porcentual de casos registrados',
                    #color_continuous_midpoint=np.average(victimas['Edad']),
                    center={"lat": 20.400417, "lon": -89.134857},
                    mapbox_style="carto-positron", 
                    zoom=7.5,
                    #opacity=0.8,
                    )
fig.update_geos(fitbounds="locations", visible=False)

#fig.update_layout(xaxis=dict(domain=[0, 0.5]), yaxis=dict(domain=[0.25, 0.75]))

fig.add_annotation(
    # The arrow head will be 25% along the x axis, starting from the left
    x=0,
    # The arrow head will be 40% along the y axis, starting from the bottom
    y=0.98,
    text="<b>Temporalidad:</b> {}-{}<br><b>Total de casos:</b> {}".format(2019,2020,total_casos),
    showarrow=False,
    bordercolor="black",
    bgcolor="white",
    borderwidth=1.5,
    opacity=0.8
)

fig.update(layout = dict(title=dict(x=0.5)))
fig.update_layout(
    margin={"r":0,"t":30,"l":10,"b":10},
    coloraxis_colorbar={
        'title':'Variación',}, 
    coloraxis_colorbar_ticksuffix='%'),
fig.update_layout(title_y=1, title_x=0)
fig.update_layout(coloraxis_showscale=True)
fig.update_coloraxes(cmax=100, cmin=-100)

#print("plotly express hovertemplate:", fig.data[0].hovertemplate)
fig.update_traces(hovertemplate='Municipio=%{location}<br>Casos 2020=%{customdata[0]}<br>Casos 2019=%{customdata[1]}<br>Variación=%{z}%<extra></extra>')
 
pio.write_html(fig, file='../docs/figs/indicador_municipal_mapa_variacion_casos.html', include_plotlyjs='cdn', full_html=False)
fig.show()

In [ ]:
fechas_groupby = subtipo.groupby(['fecha_hechos','SubtipoOrd'])['fk_euv'].count()
fechas_groupby=pd.DataFrame(fechas_groupby.reset_index())
fechas_groupby_emer = fechas_groupby[fechas_groupby.SubtipoOrd=='Emergencia']
fechas_groupby_na = fechas_groupby[fechas_groupby.SubtipoOrd=='No especificado']
fechas_groupby_prev = fechas_groupby[fechas_groupby.SubtipoOrd=='Preventiva']

In [ ]:
fechas_groupby_total = subtipo.groupby(['fecha_hechos'])['fk_euv'].count()
fechas_groupby_total=pd.DataFrame(fechas_groupby_total.reset_index())
total = pd.DataFrame(fechas_groupby_total.reset_index().groupby([pd.Grouper(freq='W', key='fecha_hechos')])['fk_euv'].sum()).reset_index()
total['var'] = round(total['fk_euv'].pct_change()*100,2)
inf = total[total['var'].isin([np.inf, -np.inf])].index
total.loc[inf,'var']=100
nan = total[total['var'].isin([np.nan])].index
total.loc[nan,'var']=0
total
total

In [ ]:
emer = pd.DataFrame(fechas_groupby_emer.reset_index().groupby([pd.Grouper(freq='W', key='fecha_hechos')])['fk_euv'].sum()).reset_index()
emer['var'] = round(emer['fk_euv'].pct_change()*100,2)
inf = emer[emer['var'].isin([np.inf, -np.inf])].index
emer.loc[inf,'var']=100
nan = emer[emer['var'].isin([np.nan])].index
emer.loc[nan,'var']=0
emer

In [ ]:
prev = pd.DataFrame(fechas_groupby_prev.reset_index().groupby([pd.Grouper(freq='W', key='fecha_hechos')])['fk_euv'].sum()).reset_index()
prev['var'] = round(prev['fk_euv'].pct_change()*100,2)
inf = prev[prev['var'].isin([np.inf, -np.inf])].index
prev.loc[inf,'var']=100
nan = prev[prev['var'].isin([np.nan])].index
prev.loc[nan,'var']=0
prev

In [ ]:
na = pd.DataFrame(fechas_groupby_na.reset_index().groupby([pd.Grouper(freq='W', key='fecha_hechos')])['fk_euv'].sum()).reset_index()
na['var'] = round(na['fk_euv'].pct_change()*100,2)
inf = na[na['var'].isin([np.inf, -np.inf])].index
na.loc[inf,'var']=100
nan = na[na['var'].isin([np.nan])].index
na.loc[nan,'var']=0
na

In [ ]:
"""fig = px.line(prev, x=prev.fecha_hechos, y="fk_euv", title='Grafica #. Variación de órdenes de proteccion por tipo', 
                markers=True,hover_data={"var": True}, labels={'fecha_hechos':'Fecha Hechos','SubtipoOrd':'Subtipo Orden','fk_euv':'Órdenes de protección'}, 
                range_x=['2015-01-01',max(semanas_groupby.fecha_hechos)], ) """

               
fig = go.Figure()

#fig.add_scatter(x=na.fecha_hechos, y=na.fk_euv, mode='lines+markers', name= 'No especificado')
#fig.add_scatter(x=emer.fecha_hechos, y=emer.fk_euv, mode='lines+markers', name='Emergencia')
#fig.add_scatter(x=prev.fecha_hechos, y=prev.fk_euv, mode='lines+markers', name='Preventiva')

fig.add_trace(go.Scatter(
        x=na.fecha_hechos,
        y=na.fk_euv,
        name='No especificado',
        text=na['var'],
        #hovertemplate=
        #"<b>%{text}</b><br><br>" +
        #"GDP per Capita: %{x:$,.0f}<br>" +
        #"Life Expectation: %{y:.0%}<br>" +
        #"Population: %{marker.size:,}" +
        #"<extra></extra>",
        #marker_size=continent['size'],
        mode='markers+lines'
        ))
fig.add_trace(go.Scatter(
        x=emer.fecha_hechos,
        y=emer.fk_euv,
        name='Emergencia',
        text=emer['var'],
        #hovertemplate=
        #"<b>%{text}</b><br><br>" +
        #"GDP per Capita: %{x:$,.0f}<br>" +
        #"Life Expectation: %{y:.0%}<br>" +
        #"Population: %{marker.size:,}" +
        #"<extra></extra>",
        #marker_size=continent['size'],
        mode='markers+lines'
        ))
fig.add_trace(go.Scatter(
        x=prev.fecha_hechos,
        y=prev.fk_euv,
        name='Preventiva',
        text=prev['var'],
        #hovertemplate=
        #"<b>%{text}</b><br><br>" +
        #"GDP per Capita: %{x:$,.0f}<br>" +
        #"Life Expectation: %{y:.0%}<br>" +
        #"Population: %{marker.size:,}" +
        #"<extra></extra>",
        #marker_size=continent['size'],
        mode='markers+lines'
        ))

fig.add_trace(go.Scatter(
        x=total.fecha_hechos,
        y=total.fk_euv,
        name='Total',
        text=total['var'],
        #hovertemplate=
        #"<b>%{text}</b><br><br>" +
        #"GDP per Capita: %{x:$,.0f}<br>" +
        #"Life Expectation: %{y:.0%}<br>" +
        #"Population: %{marker.size:,}" +
        #"<extra></extra>",
        #marker_size=continent['size'],
        mode='markers+lines'
        ))


fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            #dict(step="all")
        ])
    )
)

#print("plotly express hovertemplate:", fig.data[0].hovertemplate)
fig.update_traces(hovertemplate='Fecha hecho=%{x}<br>Órdenes registradas=%{y}<br>Variación=%{text}%<extra></extra>',)
fig.update_xaxes(range=['2015-01-01',max(semanas_groupby.fecha_hechos)])
fig.update_layout(
    title="Grafica 10. Variación de órdenes de proteccion por tipo",xaxis_title="Fecha Hechos", yaxis_title="Órdenes de Protección")
#fig.update_xaxes(type="log")
pio.write_html(fig, file='../docs/figs/indicador_municipal_timeline_subtipo.html', include_plotlyjs='cdn', full_html=False)
fig.show()